This was to create an LDA model, but the results are bad and undesirable, which I think is the result of the source data. I'm going to move forward with analyzing the data based on Counts/CountVectorizers.

*.ipynb filter=strip-notebook-output  

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
import json 
import logging
from pathlib import Path

import logging

def create_fh_logger(file):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
    file_handler = logging.FileHandler(file)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    return logger

In [ ]:
# locations of json files + a place to store a log
src = Path.cwd().parent.parent / 'processing' / 'nro_declassified' / 'tokenized'
dst = src.parent / 'topics'
files = list(src.glob('*json'))
logs = Path.cwd() / 'logs'
logs.mkdir(exist_ok=True)
logger = create_fh_logger(logs / "topic_model.log")

In [ ]:
documents = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
    words = [item['word'] for item in data]
    documents.append(preprocess_string((' ').join(words))) # some edge cases I still missed, letting gensim pick up the slack here

In [ ]:
dictionary = Dictionary(documents)
dictionary.filter_extremes(no_below=0.1, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus=corpus, id2word=dictionary, per_word_topics=True,  num_topics=6, passes=5)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

In [ ]:
for document_num, doc in enumerate(corpus):
    doc_topics = lda_model.get_document_topics(doc)
    # Optionally, sort the topics by their contribution to this document
    doc_topics_sorted = sorted(doc_topics, key=lambda x: x[1], reverse=True)
    for topic_num, prop_topic in doc_topics_sorted:
        # Print the top topic and its contribution
        if topic_num == 4:
            print(f"Document {document_num} Top Topics: {doc_topics}")
            print(f'File: {files[document_num]}')
            print(f"Top topic: {topic_num}, Contribution: {prop_topic:.4f}")
            # Print top words for this topic
            word_contributions = lda_model.show_topic(topic_num)
            print(f"Top words for top topic: {word_contributions}")
            print("\n")